December 5, 2023
1. Use Read1s for RPTR; Use assembled reads for AD
2. Using a Tile-BC combination, I can find the RPTRbc it's associated with and see if the presence of one bc properly predicts the presence of the other bc
3. Goals: 
Correlation between two RPTR runs from same time point (maybe from different samples)
RPTR BC abundance to  AD BC abundance ratios ― maybe a histogram of ratios
Compare against existing data: For those with high RPTR:AD ratio (ie expected high activity) are they also high activity in Jordan’s data


In [1]:
import datetime
import time
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv

# import duckdb

# conn = duckdb.connect('/global/scratch/users/empchase/Seq_analysis/mydb.db') # create an in-memory database

In [2]:
conn.sql("SHOW ALL TABLES")

┌──────────┬─────────┬─────────────┬───────────────────────┬───────────────────────────────────────────────┬───────────┐
│ database │ schema  │    name     │     column_names      │                 column_types                  │ temporary │
│ varchar  │ varchar │   varchar   │       varchar[]       │                   varchar[]                   │  boolean  │
├──────────┼─────────┼─────────────┼───────────────────────┼───────────────────────────────────────────────┼───────────┤
│ mydb     │ main    │ SI_VI_AD0   │ [index, BCs, TBB, T…  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]           │ false     │
│ mydb     │ main    │ SI_VI_AD1   │ [index, BCs, TBB, T…  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]           │ false     │
│ mydb     │ main    │ SI_VI_RPTR0 │ [index, BCs, TBB, T…  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]           │ false     │
│ mydb     │ main    │ SI_VI_RPTR1 │ [index, BCs, TBB, T…  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]           │ false     │
│ mydb     │ main    │ TBB      

In [3]:
pwd

'/global/scratch/users/empchase/CiberVII/CiberVII_CZB'

In [2]:
#where are the reads
adpath = '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/'
rptrpath = '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/EC_Demultiplexed/'

addirectory = os.fsencode(adpath)
rptrdirectory = os.fsencode(rptrpath)

AD1files = []

for file in os.listdir(addirectory):
    filename = os.fsdecode(file)
#     x = filename.split('_')
#     filename = path + filename
    
    if filename.endswith('.fastq'): 
        x = filename.split('_')
        filename = adpath + filename
#         print(x)
        if 'AD' in x:
            AD1files.append(filename)
            
            
RPTR1files = [] #nt: one file ends with .txt, and is not in fastq format, so I have to manually input that into analysis later

for file in os.listdir(rptrdirectory):
    filename = os.fsdecode(file)
    if filename.endswith('.fastq'): #or filename.endswith('.txt')
        x = filename.split('_')
        filename = rptrpath + filename
        if 'RPTR' in x:
            RPTR1files.append(filename)
            
print(AD1files)
print(RPTR1files)

['/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/AD_4_30_16X16_0927_S205_L003_R1_001_demultiplexed.assembled.fastq', '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.fastq', '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/AD_4_15_20X20_S8_L004.fastq.gz.assembled.fastq', '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/AD_4_30_16X16_0925_S12_L004.fastq.gz.assembled.fastq', '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/AD_4_30_30X20_S206_L003_R1_001_demultiplexed.assembled.fastq', '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/AD_3_30_16X16_S13_L004.fastq.gz.assembled.fastq', '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/assembledreads/EC/AD_3_30_30X20_S206_L004_R1_001_demultiplexed.assembled.fastq']
['/global/scratch/users/empchase/CiberVII/CiberVII_CZB/EC_Demultiplexed/RPTR_maxiprep_20X20_S1

In [3]:
len(AD1files)

7

In [3]:
#get list of designed tiles
a10design = '/global/scratch/users/empchase/A10_sequencing/analysis/a10_designfile.csv'

a10dt = []

with open(a10design, 'r') as f:
    for line in f:
        if "ArrayDNA" in line:
            pass
        else:
            a10dt.append(line.strip())
# print(a10dt[:5])
print(len(a10dt))#20783

#create a dictionary of values
dtiles = {}
for i in a10dt:
    dtiles[i] = 1
    
# print(d)

20783


In [4]:
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N':'N', 'X':'X'} 
    bases = list(seq) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)
def reverse_complement(s):
        return complement(s[::-1])

In [5]:
#putative consensus sequences ***reverse complement of snapgene***
adp = 'CGGGCCC'#7 bp ; beforeAD barcode in read1
adf = 'GGCGCGC' #7bp ; after AD barcode in read1

rpp = 'AGCGGCC' #7bp ; before rptr barcode in read1
rpf = 'CTCGAGT' #7 bp ; after rptr barcode in read1

tp = "TAGTCA"
tf = "GCTAGC"

In [6]:
def getmid(seq, pre, post, bclen):
    # seq = the sequence to parse
    # pre = substring that precedes piece of interest
    # post = substring that follows piece of interest
    # returns piece of interest

    re_key = pre + "(.*)"+ post 
    poi_search = re.search(re_key, seq)
#     print(poi_search)
#     poi = ''


    if poi_search is None:
        #the barcode will be called X
        poi = "X"
        
        #then we search for which restriction site had the error
        #regex for the bc we want to ignore
        w = "(.{"+str(bclen)+"})" 
#         print(w)
        pre_re = pre + w + "(.{7})"
        pre_search = re.search(pre_re, seq)
#         print(pre_search)
        post_re = "(\w{7})" + w + post
        post_search = re.search(post_re, seq)
#         print(post_search)
        
        if pre_search is None and post_search is None:
            a = 'X'
            z = 'X'
        elif pre_search is None:
            poi = post_search.group(2)
            a = post_search.group(1)
            z = post
        elif post_search is None:
            poi = pre_search.group(1)
            z = pre_search.group(2)
            a = pre
        else:
            a = "Z"
            z = 'Z'
#             print(seq)
            
    else:
        poi = poi_search.group(1)
        a = pre
        z = post
    
    return poi, a, z



# function that just looks for bc
def rbc_finder(readfile, bc_pre=rpp, bc_post=rpf, bc_len=14):

    # make lists of reads
    seqlist = []
    
    with open(readfile, 'r') as fin:
        for line in fin:
            if line.startswith('@'):
                #look at next line to get read sequence, add to list
                seq = next(fin)
                seq = seq.strip()
                seqlist.append(seq)

    
    #make lists of BCs from list of reads
    bc_list = []
    bc_lens = []
#     q_list = []
    
#     prex_list = []
#     prex_len = []
#     postx_list = []
#     postx_len = []
       
    for read in seqlist:
   
        bc, prex, postx = getmid(read, bc_pre, bc_post, bc_len)
        bc = reverse_complement(bc) #return reverse complement
        bc_list.append(bc)
        bcl = len(bc)
        bc_lens.append(bcl)
        
#         prex_list.append(prex)
#         prex_len.append(len(prex))
#         postx_list.append(postx)
#         postx_len.append(len(postx))
#         if bcl == bc_len:
#             q_list.append(1)
#         else:
#             q_list.append(0)
    #         print(bc)

    
    BC_dict = {"BCs":bc_list, "Length":bc_lens #,
#                "Qual":q_list, "prex":prex_list, 'prexl':prex_len,
#                'postx':postx_list, 'postxl':postx_len
              } 
    BC_df = pd.DataFrame.from_dict(BC_dict)
    
    #label BC with library name
    
    libname = '_'.join(readfile.split('/')[-1].split('_')[0:6])
    BC_df['Library'] = libname
    
    return BC_df

In [7]:
def adbc_mapper(readfile, tile_pre = tp, tile_post = tf, 
                  adBC_pre = adp, adBC_post = adf):
    
    #readfile = file of SEQUENCES ONLY
    # *_pre or *_post = the consensus sequences before or after each feature

    # make lists of reads
    readlist = []

    
    with open(readfile, 'r') as fin:
        for line in fin:
            if line.startswith('@'):
                #look at next line to get read sequence, add to list
                seq = next(fin)
                seq = seq.strip()
                readlist.append(seq)
                
    #make lists of tiles/BCs from list of reads
    tile_list = []
    tile_lengths= []
    
    des_query = [] # tells us if tile matches design or not
    
    adBC_list = []
    adBC_lengths = []
    
    tile_adbc = []
   
    
    for read in readlist:
        tile, tpre, tpost = getmid(read, tile_pre, tile_post, 120) #use consensus seq to find tile
        tile = reverse_complement(tile)
        tile_list.append(tile) #add tile to list
        tile_len = len(tile) #find length of tile
        tile_lengths.append(tile_len) #add length to list
            
        if tile in dtiles:
            des_query.append(1)
        else:
            des_query.append(0)
#         print(tile)
        
        adBC, prea, posta = getmid(read, adBC_pre, adBC_post, 11)
        adBC = reverse_complement(adBC)
        adBC_list.append(adBC)
        adBC_len = len(adBC)
        adBC_lengths.append(adBC_len)
        
        tile_adbc.append(tile+'-'+adBC)

            
    # make the df
    
    tileBC_dict = {"Tiles":tile_list, "T Len" : tile_lengths, "Designed": des_query, 
                  "AD BCs":adBC_list, "A Len": adBC_lengths, 'Tile-AD': tile_adbc}
    tileBC_df = pd.DataFrame.from_dict(tileBC_dict)
    
    libname = '_'.join(readfile.split('/')[-1].split('_')[0:6])
    tileBC_df['Library'] = libname
    
    return tileBC_df

In [8]:
# function that just looks for bc
def bc_finder_seqonly(readfile,bc_pre, bc_post, bc_len):

    # make lists of reads
    seqlist = []
    
    with open(readfile, 'r') as fin:
        for line in fin:
            seq = line.strip()
            seqlist.append(seq)

    
    #make lists of BCs from list of reads
    bc_list = []
    bc_lens = []
#     q_list = []
    
#     prex_list = []
#     prex_len = []
#     postx_list = []
#     postx_len = []
       
    for read in seqlist:
   
        bc, prex, postx = getmid(read, bc_pre, bc_post, bc_len)
        bc = reverse_complement(bc) #return reverse complement
        bc_list.append(bc)
        bcl = len(bc)
        bc_lens.append(bcl)
        
#         prex_list.append(prex)
#         prex_len.append(len(prex))
#         postx_list.append(postx)
#         postx_len.append(len(postx))
#         if bcl == bc_len:
#             q_list.append(1)
#         else:
#             q_list.append(0)
#     #         print(bc)

    
    BC_dict = {"BCs":bc_list, "Length":bc_lens
#                ,"Qual":q_list, 
#                "prex":prex_list, 'prexl':prex_len,
#                'postx':postx_list, 'postxl':postx_len
              } #not lengths but matches to rpp and rpf
    BC_df = pd.DataFrame.from_dict(BC_dict)
    
    #label BC with library name
    
    libname = '_'.join(readfile.split('/')[-1].split('_')[0:6])
    BC_df['Library'] = libname
    
    return BC_df

In [10]:
#read in as df and create dictionary
tbbdf = conn.sql("""SELECT DISTINCT * FROM TBB""").df() #maybe do select distinct next time
tbbdf.head()

,Tiles,ADbc,RPTRbc,TBBs,TILEADBC
0,AAACCAGTTTCTTCTCCAGCTAGATCTCCAACTGCTGCTCCAAGAT...,AACGATTATGT,ATCGGTTGTAGTAA,AAACCAGTTTCTTCTCCAGCTAGATCTCCAACTGCTGCTCCAAGAT...,AAACCAGTTTCTTCTCCAGCTAGATCTCCAACTGCTGCTCCAAGAT...
1,AATATGTCTAGACCAACATTTCCAGAAGGTGGTGTTTCTAGAGCTT...,ATACAACGTCA,AAGTTGTTTTATCT,AATATGTCTAGACCAACATTTCCAGAAGGTGGTGTTTCTAGAGCTT...,AATATGTCTAGACCAACATTTCCAGAAGGTGGTGTTTCTAGAGCTT...
2,TTTAATATTCATTCTTCTGTCTTGGATTCTGTCTTCTCTTCTACTA...,AACCATTGGAA,GATCGGTAGGTCAC,TTTAATATTCATTCTTCTGTCTTGGATTCTGTCTTCTCTTCTACTA...,TTTAATATTCATTCTTCTGTCTTGGATTCTGTCTTCTCTTCTACTA...
3,TTCTGGAATATTATTGTTTCTGTTAGATTGTGTCCAATTATTCCAG...,CCATGTTGAGC,TGGCAACACCGACA,TTCTGGAATATTATTGTTTCTGTTAGATTGTGTCCAATTATTCCAG...,TTCTGGAATATTATTGTTTCTGTTAGATTGTGTCCAATTATTCCAG...
4,GGTCATAGAGCTGGTGAACCAAATAAAGCTGTTATTTGTTCTTTGG...,ACAAAAATCTT,TACGCGCTGTCCCC,GGTCATAGAGCTGGTGAACCAAATAAAGCTGTTATTTGTTCTTTGG...,GGTCATAGAGCTGGTGAACCAAATAAAGCTGTTATTTGTTCTTTGG...


In [13]:
tbbdf.loc[0, 'TBBs'] #'GGTGTTACTGCTGCTGATGCTGAATCTTTGGTTGCTGATCACTTCTTGGCTTCTTCTGCTGATTCTACTCCATTGTTTGAATTGGATGCTGTTGAATCTGATCCACAAGGTTGGAACTCT-ACCTCAAAGGC-CCGCTCATAGTTTT'

'GGTATTTCTAAATCTAGAAGAAGGAAAGGTACTTTGCCACCAATTGCTGTTGATCCAAATGATAAGATTGCTTTGAAGAGAGCTAGAAATACTTTGGCTGCTAGAGAATCTAGACAAAGA-CCTGCTTTAAG-TGGACAAAGGCATG'

In [12]:
t1r = rbc_finder(RPTR1files[-2], rpp, rpf, 14)
t1r

,BCs,Length,Library
0,GTGAACGTTCCTCA,14,RPTR_4_15_20X20_S10_L004
1,GTGAACGTTCCTCA,14,RPTR_4_15_20X20_S10_L004
2,TCTGAATCCATCAA,14,RPTR_4_15_20X20_S10_L004
3,TGGAGAAAGAAAAT,14,RPTR_4_15_20X20_S10_L004
4,GTGAACGTTCCTCA,14,RPTR_4_15_20X20_S10_L004
...,...,...,...
1908924,GTGAACGTTCCTCA,14,RPTR_4_15_20X20_S10_L004
1908925,TGGAGAAAGAAAAT,14,RPTR_4_15_20X20_S10_L004
1908926,X,1,RPTR_4_15_20X20_S10_L004
1908927,X,1,RPTR_4_15_20X20_S10_L004


In [13]:
t1r.describe()

,Length
count,1.908929e+06
mean,9.330658e+00
std,6.240187e+00
min,1.000000e+00
25%,1.000000e+00
50%,1.400000e+01
75%,1.400000e+01
max,1.050000e+02


In [9]:
t1a = adbc_mapper(AD1files[1])
t1a

,Tiles,T Len,Designed,AD BCs,A Len,Tile-AD,Library
0,TCTCCATCTGCTCCACCATCTACTTCT,27,0,TCACCCACGTA,11,TCTCCATCTGCTCCACCATCTACTTCT-TCACCCACGTA,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
1,GAATTGTGGGTTGATGCTTTGGCTCACTTTGATCCATCTAATGAAA...,119,0,TCGATCTGCGG,11,GAATTGTGGGTTGATGCTTTGGCTCACTTTGATCCATCTAATGAAA...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
2,TCATTTGGTTATATGACTACTTCTTCTGTTCCAGTTCCACCAGGTA...,120,1,GCAGCATGAAA,11,TCATTTGGTTATATGACTACTTCTTCTGTTCCAGTTCCACCAGGTA...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
3,AGACAACCAAATAGATCTGTTTCTCAACCATCTGCTAATGGTGGTG...,120,1,ACACGCTGCCG,11,AGACAACCAAATAGATCTGTTTCTCAACCATCTGCTAATGGTGGTG...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
4,AGAAGAAGAGATAAACCATTGCCACCAATTGTTGTTGAAGATCCAT...,120,1,TTGAGACATGC,11,AGAAGAAGAGATAAACCATTGCCACCAATTGTTGTTGAAGATCCAT...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
...,...,...,...,...,...,...,...
7338103,ACTATGCCATTGTCTCATGACTTTGAAGATGCTCATCCAGATACTA...,120,1,TAACTGACATG,11,ACTATGCCATTGTCTCATGACTTTGAAGATGCTCATCCAGATACTA...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
7338104,GCTGCTGTTGCTATTATTGTTGGTGCTGTTTCTTTGTCTTTGGGTA...,120,1,GCCAAATATAC,11,GCTGCTGTTGCTATTATTGTTGGTGCTGTTTCTTTGTCTTTGGGTA...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
7338105,ATGGGTACTGTTTCTCCACAAGATTTGTTTGTTAGAGATTCATTTG...,105,0,AATTTAGTCCT,11,ATGGGTACTGTTTCTCCACAAGATTTGTTTGTTAGAGATTCATTTG...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
7338106,CAACAACAACAACAACAACAACAACAACAACAACAACAACAACAAC...,120,1,TTCGATTTGTA,11,CAACAACAACAACAACAACAACAACAACAACAACAACAACAACAAC...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...


In [10]:
t1a.describe()

,T Len,Designed,A Len
count,7.338108e+06,7.338108e+06,7.338108e+06
mean,1.047166e+02,4.422173e-01,1.098001e+01
std,3.298164e+01,4.966500e-01,5.670403e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.090000e+02,0.000000e+00,1.100000e+01
50%,1.200000e+02,0.000000e+00,1.100000e+01
75%,1.200000e+02,1.000000e+00,1.100000e+01
max,2.270000e+02,1.000000e+00,2.550000e+02


In [14]:
t2a = adbc_mapper(AD1files[0])
t2a

,Tiles,T Len,Designed,AD BCs,A Len,Tile-AD,Library
0,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
1,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
2,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
3,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
4,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
...,...,...,...,...,...,...,...
463,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
464,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
465,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205
466,X,1,0,X,1,X-X,AD_4_30_16X16_0927_S205


In [15]:
t2a.describe()

,T Len,Designed,A Len
count,468.000000,468.000000,468.000000
mean,2.525641,0.002137,1.085470
std,13.401771,0.046225,0.921526
min,1.000000,0.000000,1.000000
25%,1.000000,0.000000,1.000000
50%,1.000000,0.000000,1.000000
75%,1.000000,0.000000,1.000000
max,120.000000,1.000000,11.000000


In [11]:
def analyze_bcs (df, bc_len, bcdict):
    # df = barcode containing df, parsed from fastq
    # bc_len = int, expected barcode length
    # tbb_dictkey = str, either 'ADbc' or 'RPTRbc'
    
    print(df.loc[0,'Library'])
    print()
    print('total reads')
    tr = df.shape[0]
    print(tr) #how many total reads
    
    cls = df[df['Length']== bc_len] #cl = correct length
    print('BCs of correct length')
    clcount = cls.shape[0]
    print(clcount)
    
    print('% BCs of correct length')
    clpct = cls.shape[0]/df.shape[0]
    print (clpct)
    
    #df of BC coverage
    cl_covdf = cls['BCs'].value_counts().to_frame().reset_index() 
    print('# Unique BCs')
    uniqbccount = cl_covdf.shape[0]
    print(uniqbccount)
#     print('SUM Unique BCs')
#     print(cl_covdf.sum(numeric_only=True)['BCs'])

    #copy down unique BCs into a list
    bcs = cl_covdf['index'].tolist()
    


    matchlist = [] #list of TBBs that match the BCs
#     matches = 0
    for x in bcs:
        try:
            tbb = bcdict[x]
        except KeyError:
            matchlist.append(0)
        else:
            matchlist.append(tbb)
#             matches += 1
            

            
    cl_covdf['TBB'] = matchlist
    
    matchesonly = cl_covdf.replace(0, np.nan)
    matchesonly = matchesonly.dropna()
    totmatches = matchesonly.sum(numeric_only=True)['BCs']
    print('# BC matches to A10 deep seq map')
    mapmatches = matchesonly.shape[0]
    print(mapmatches)
    
    print('TOT BC matches to A10 deep seq map')
    print(totmatches)
    
#     print('% unique BCs matched')
#     print(totmatches/cl_covdf.shape[0])
    
    
    print()
    
    #label the df with library name
    
    libname = df.loc[0,'Library']
    cl_covdf['Library'] = libname
    
    statslist = [libname, tr, clcount, clpct, uniqbccount, mapmatches, totmatches]
    
    
    return cl_covdf, statslist
    

In [12]:
#dictionary ~ {RPTRbc: TBB}

RPTRdict = dict(zip(tbbdf.RPTRbc, tbbdf.TBBs))

In [13]:
# #dictionary ~ {ADbc: TBB}

A10ADdict = dict(zip(tbbdf.TILEADBC, tbbdf.TBBs)) #this function overwrites the value of any repeated key. 

In [35]:
#Test
# print(Rlist[2])
shorttest, st_stats = analyze_bcs(tmp, 14, RPTRdict)
print(st_stats)
shorttest.head()

RPTR_4_30_16X16_sequences_R1

total reads
2364670
BCs of correct length
1604
% BCs of correct length
0.0006783187506079072
# Unique BCs
359
# BC matches to A10 deep seq map
29
TOT BC matches to A10 deep seq map
53

['RPTR_4_30_16X16_sequences_R1', 2364670, 1604, 0.0006783187506079072, 359, 29, 53]


,index,BCs,TBB,Library
0,ACCGCGGTATCAGA,150,0,RPTR_4_30_16X16_sequences_R1
1,TTCAGACGTGTGCT,144,0,RPTR_4_30_16X16_sequences_R1
2,AGAGTGGACCGCGG,117,0,RPTR_4_30_16X16_sequences_R1
3,ACCGCGTGTATGCC,112,0,RPTR_4_30_16X16_sequences_R1
4,AGTGGACCGCGCGG,99,0,RPTR_4_30_16X16_sequences_R1


It works ! When I come back try all:

In [14]:
# make df out of each fastq file
Alist = []
Rlist = []

for file in AD1files:
    print('file opened', datetime.datetime.now())
    x = adbc_mapper(file)
    Alist.append(x)


for file in RPTR1files:
    print('file opened', datetime.datetime.now())
    x = rbc_finder(file, rpp, rpf, 14)
    Rlist.append(x)

file opened 2023-12-19 10:42:44.170870
file opened 2023-12-19 10:42:44.263771


KeyboardInterrupt: 

In [17]:
#investigate a few of interest:
print('file opened', datetime.datetime.now())

mp = rbc_finder(rptrpath+'RPTR_maxiprep_20X20_S11_L004_R1_001.fastq', rpp, rpf, 14)

print('file opened', datetime.datetime.now())
goodprep = rbc_finder(rptrpath+'RPTR_3_30_16X16_S15_L004_R1_001.fastq', rpp, rpf, 14)

file opened 2023-12-19 10:52:51.906446
file opened 2023-12-19 10:55:51.489572


In [18]:
lil_list = [mp, goodprep]

In [ ]:
# Rlist.pop(6)
len(Rlist)

In [ ]:
tmp = bc_finder_seqonly(rptrpath+'RPTR_4_30_16X16_sequences_R1_001.txt', rpp, rpf, 14)

In [ ]:
Rlist.append(tmp)

In [19]:
def split_TBB (countsdf):
    # finds the putative tiles and ADs via the TBB and adds it to the df IN PLACE
    putative_tiles = []
    putative_ads = []

    for index, row in countsdf.loc[:, ['TBB']].iterrows():
        try:
            #     print(row['TBB'])
            tbb_str = row['TBB']
            ptile = tbb_str.split('-')[0]
            pad = tbb_str.split('-')[1]
        except AttributeError:
            ptile = 0
            pad = 0 
        finally:
            putative_tiles.append(ptile)
            putative_ads.append(pad)

    countsdf['Putative_Tile'] = putative_tiles
    countsdf['Putative_AD'] = putative_ads

In [ ]:
#analyze all dfs and get dictionary of ADs:TBBs from each df
#the dfs we keep in the end in analyzed_rptr_dfs are not clean (have 0s and RPTR BCs not matching to A10)

analyzed_rptr_dfs = []
ADdict = {}
rptrstats = []

for og_df in Rlist:
    # get the counts dfs, split the TBBs in each
    counts_df, statslist = analyze_bcs(og_df, 14, RPTRdict)
    rptrstats.append(statslist)
#     print(len(rptrstats))
    split_TBB(counts_df)
    analyzed_rptr_dfs.append(counts_df)
    
    counts_df_clean = counts_df.replace(0, np.nan)
    counts_df_clean = counts_df_clean.dropna()
    ad_dict = dict(zip(counts_df_clean.Putative_AD, counts_df_clean.TBB))
    ADdict.update(ad_dict)

In [ ]:
#analyze all dfs and get dictionary of ADs:TBBs from each df
#the dfs we keep in the end in analyzed_rptr_dfs are not clean (have 0s and RPTR BCs not matching to A10)

analyzed_rptr_dfs = []
ADdict = {}
rptrstats = []

for og_df in lil_list:
    # get the counts dfs, split the TBBs in each
    counts_df, statslist = analyze_bcs(og_df, 14, RPTRdict)
    rptrstats.append(statslist)
#     print(len(rptrstats))
    split_TBB(counts_df)
#     analyzed_rptr_dfs.append(counts_df)
    
    counts_df_clean = counts_df.replace(0, np.nan)
    counts_df_clean = counts_df_clean.dropna()
    analyzed_rptr_dfs.append(counts_df_clean)
#     ad_dict = dict(zip(counts_df_clean.Putative_AD, counts_df_clean.TBB))
#     ADdict.update(ad_dict)

In [ ]:
rstats_tocsv = pd.DataFrame(rptrstats, columns = [
    'Library', 'TotalReads', 'Reads_correctlen', 'Reads_clpct', 'TotBC_Unique','MatchesCounts_Unique', 'MatchesCounts_Total'])

rstats_tocsv.head() #this will tell us if TBB df has changed with distinct    

In [ ]:
len(ADdict) #6040 in TBB_Union (the misnomer); 9409 in TBB (the real union table)

In [41]:
testing = Alist[1]
testing[(testing["A Len"]==11) & (testing['T Len'] == 120)]

,Tiles,T Len,Designed,AD BCs,A Len,Tile-AD,Library
2,TCATTTGGTTATATGACTACTTCTTCTGTTCCAGTTCCACCAGGTA...,120,1,GCAGCATGAAA,11,TCATTTGGTTATATGACTACTTCTTCTGTTCCAGTTCCACCAGGTA...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
3,AGACAACCAAATAGATCTGTTTCTCAACCATCTGCTAATGGTGGTG...,120,1,ACACGCTGCCG,11,AGACAACCAAATAGATCTGTTTCTCAACCATCTGCTAATGGTGGTG...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
4,AGAAGAAGAGATAAACCATTGCCACCAATTGTTGTTGAAGATCCAT...,120,1,TTGAGACATGC,11,AGAAGAAGAGATAAACCATTGCCACCAATTGTTGTTGAAGATCCAT...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
5,ATTCCAACTACTACTCCAGAATCTTCATTCTTGGAACCATTAAATT...,120,1,CCGCCTCGCCA,11,ATTCCAACTACTACTCCAGAATCTTCATTCTTGGAACCATTAAATT...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
6,TTGGATTTGTTTGATTTGCCAACTCAATCTCCATTGTTGGATGATT...,120,1,CTTGATCCGAC,11,TTGGATTTGTTTGATTTGCCAACTCAATCTCCATTGTTGGATGATT...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
...,...,...,...,...,...,...,...
7338102,ATGGCTTCTTTGCAATTTATTGAACCAGCTGGTACTATTTCTACTT...,120,0,GGTCAAATCAT,11,ATGGCTTCTTTGCAATTTATTGAACCAGCTGGTACTATTTCTACTT...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
7338103,ACTATGCCATTGTCTCATGACTTTGAAGATGCTCATCCAGATACTA...,120,1,TAACTGACATG,11,ACTATGCCATTGTCTCATGACTTTGAAGATGCTCATCCAGATACTA...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
7338104,GCTGCTGTTGCTATTATTGTTGGTGCTGTTTCTTTGTCTTTGGGTA...,120,1,GCCAAATATAC,11,GCTGCTGTTGCTATTATTGTTGGTGCTGTTTCTTTGTCTTTGGGTA...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...
7338106,CAACAACAACAACAACAACAACAACAACAACAACAACAACAACAAC...,120,1,TTCGATTTGTA,11,CAACAACAACAACAACAACAACAACAACAACAACAACAACAACAAC...,AD_maxiprep_20X20_S9_L004.fastq.gz.assembled.f...


In [42]:
testing.loc[0,'Tile-AD']

'TCTCCATCTGCTCCACCATCTACTTCT-TCACCCACGTA'

In [23]:
mykey = 'GGTGTTACTGCTGCTGATGCTGAATCTTTGGTTGCTGATCACTTCTTGGCTTCTTCTGCTGATTCTACTCCATTGTTTGAATTGGATGCTGTTGAATCTGATCCACAAGGTTGGAACTCT-ACCTCAAAGGC'
mykeys = mykey.split('-')
aktest = mykeys[1]
tktest = mykeys[0]
myquery = conn.sql("SELECT RPTRbc FROM TBB WHERE ADBC='{}' AND TILES='{}'".format(aktest, tktest)).to_df() #("SELECT RPTRBC FROM TBB WHERE ADBC={adfill} AND TILES={tilefill}".format(adfill=aktest, tilefill=tktest))
myquery.loc[0,'RPTRbc'] #errors as expected:)

'CCGCTCATAGTTTT'

In [20]:
def analyze_tiles_adbcs (df, bc_len, bcdict):
    # df = barcode containing df, parsed from fastq
    # bc_len = int, expected barcode length
    # tbb_dictkey = str, either 'ADbc' or 'RPTRbc'
    
    print(df.loc[0,'Library'])
    print()
    print('total reads')
    tr = df.shape[0]
    print(tr) #how many total reads
    
    cls = df[df['Length']== bc_len] #cl = correct length
    print('BCs of correct length')
    clcount = cls.shape[0]
    print(clcount)
    
    print('% BCs of correct length')
    clpct = cls.shape[0]/df.shape[0]
    print (clpct)
    
    #df of BC coverage
    cl_covdf = cls['BCs'].value_counts().to_frame().reset_index() 
    print('# Unique BCs')
    uniqbccount = cl_covdf.shape[0]
    print(uniqbccount)
#     print('SUM Unique BCs')
#     print(cl_covdf.sum(numeric_only=True)['BCs'])

    #copy down unique BCs into a list
    bcs = cl_covdf['index'].tolist()
    


    matchlist = [] #list of TBBs that match the BCs
#     matches = 0
    for x in bcs:
        try:
            tbb = bcdict[x]
        except:
            matchlist.append(0)
        else:
            matchlist.append(tbb)
#             matches += 1
            

            
    cl_covdf['TBB'] = matchlist
    
    matchesonly = cl_covdf.replace(0, np.nan)
    matchesonly = matchesonly.dropna()
    totmatches = matchesonly.sum(numeric_only=True)['BCs']
    print('# BC matches to A10 deep seq map')
    mapmatches = matchesonly.shape[0]
    print(mapmatches)
    
    print('TOT BC matches to A10 deep seq map')
    print(totmatches)
    
#     print('% unique BCs matched')
#     print(totmatches/cl_covdf.shape[0])
    
    
    print()
    
    #label the df with library name
    
    libname = df.loc[0,'Library']
    cl_covdf['Library'] = libname
    
    statslist = [libname, tr, clcount, clpct, uniqbccount, mapmatches, totmatches]
    
    
    return cl_covdf, statslist
    

In [21]:
# def analyze_tiles_adbcs (df, bc_len=11, tile_len=120):
#     # df = barcode containing df, parsed from fastq
#     # bc_len = int, expected barcode length
#     # tbb_dictkey = str, either 'ADbc' or 'RPTRbc'
    
#     print(df.loc[0,'Library'])
#     print()
#     print('total reads')
#     tr = df.shape[0]
#     print(tr) #how many total reads
    
#     cls = df[(df['A Len']== bc_len) & (df['T Len']==tile_len)] #cl = correct length
#     print('Reads w BC and Tile of correct length')
#     clcount = cls.shape[0]
#     print(clcount)
    
#     print('% Reads w correct length Tiles/BCs')
#     clpct = cls.shape[0]/df.shape[0]
#     print (clpct)
    
#     #df of BC coverage
#     cl_covdf = cls['Tile-AD'].value_counts().to_frame().reset_index() 
#     print('# Unique Tile-ADbc pairs')
#     uniqbccount = cl_covdf.shape[0]
#     print(uniqbccount)
# #     print('SUM Unique BCs')
# #     print(cl_covdf.sum(numeric_only=True)['BCs'])

#     #copy down unique BCs into a list
#     bcs = cl_covdf['index'].tolist()
    


#     matchlist = [] #list of TBBs that match the BCs
#     print('mathlist initiated', datetime.datetime.now())
#     for x in bcs:
#         try:
#             both = x.split('-')
#             ak = both[1]
#             tk = both[0]
#             myquery = conn.sql("SELECT RPTRbc FROM TBB WHERE ADBC='{}' AND TILES='{}'".format(ak, tk)).to_df() #try ("SELECT RPTRBC FROM TBB WHERE ADBC={adfill} AND TILES={tilefill}".format(adfill=ak, tilefill=tk))
#             pr = myquery.loc[0,'RPTRbc'] #pr as in putative reporter
#         except KeyError:
#             matchlist.append(0)
#         else:
#             matchlist.append(pr)
    
#     print('mathlist finished', datetime.datetime.now())

    
            

            
#     cl_covdf['PutativeRPTR'] = matchlist
    
#     matchesonly = cl_covdf.replace(0, np.nan)
#     matchesonly = matchesonly.dropna()
#     totmatches = matchesonly.sum(numeric_only=True)['Tile-AD']
#     print('# BC matches to A10 deep seq map')
#     mapmatches = matchesonly.shape[0]
#     print(mapmatches)
    
#     print('TOT BC matches to A10 deep seq map')
#     print(totmatches)
    
# #     print('% unique BCs matched')
# #     print(totmatches/cl_covdf.shape[0])
    
    
#     print()
    
#     #label the df with library name
    
#     libname = df.loc[0,'Library']
#     cl_covdf['Library'] = libname
    
#     statslist = [libname, tr, clcount, clpct, uniqbccount, mapmatches, totmatches]
    
    
#     return cl_covdf, statslist
    

In [22]:
testdf_ad, teststats_ad = analyze_tiles_adbcs(Alist[1], 11, A10ADdict)
print(teststats_ad)
testdf_ad.head()

IndexError: list index out of range

In [ ]:
#analyze AD BCs
analyzed_ad_dfs = []
counter_ADdict = {}

for og_df in Alist:
    # get the counts dfs, split the TBBs in each
    counts_df = analyze_bcs(og_df, 11, ADdict)
    split_TBB(counts_df)
    analyzed_ad_dfs.append(counts_df)
    
    counts_df_clean = counts_df.replace(0, np.nan)
    counts_df_clean = counts_df_clean.dropna()
    ad_dict = dict(zip(counts_df_clean.Putative_AD, counts_df_clean.TBB))
    counter_ADdict.update(ad_dict)

In [28]:
#analyze AD BCs
A10analyzed_ad_dfs = []
counter_ADdict_A10 = {}

for og_df in Alist:
    # get the counts dfs, split the TBBs in each
    counts_df = analyze_bcs(og_df, 11, A10ADdict)
    split_TBB(counts_df)
    A10analyzed_ad_dfs.append(counts_df)
    
    counts_df_clean = counts_df.replace(0, np.nan)
    counts_df_clean = counts_df_clean.dropna()
    ad_dict = dict(zip(counts_df_clean.Putative_AD, counts_df_clean.TBB))
    counter_ADdict_A10.update(ad_dict)

AD_4_30_16X16_0925_S12

total reads
25195610
BCs of correct length
23626165
% BCs of correct length
0.9377095851221701
# Unique BCs
5495
# BC matches to A10 deep seq map
5147
TOT BC matches to A10 deep seq map
23612898

AD_4_30_30X20_S206_L003

total reads
12475045
BCs of correct length
12431038
% BCs of correct length
0.9964723974943578
# Unique BCs
1372
# BC matches to A10 deep seq map
1262
TOT BC matches to A10 deep seq map
12428562

AD_4_15_20X20_S8_L004

total reads
4067651
BCs of correct length
4059197
% BCs of correct length
0.9979216506037514
# Unique BCs
2224
# BC matches to A10 deep seq map
2094
TOT BC matches to A10 deep seq map
3952494

AD_maxiprep_20X20_S9_L004_R1

total reads
7353104
BCs of correct length
7294463
% BCs of correct length
0.9920250005983867
# Unique BCs
305788
# BC matches to A10 deep seq map
290462
TOT BC matches to A10 deep seq map
6965551

AD_4_30_16X16_0927_S205

total reads
1251
BCs of correct length
5
% BCs of correct length
0.003996802557953637
# Uni

In [29]:
#save useful tables into database

# conn.sql("CREATE TABLE SI_VI_RPTR0 AS SELECT * FROM rptrmatch0")
# conn.sql("CREATE TABLE SI_VI_RPTR1 AS SELECT * FROM rptrmatch1")
# conn.sql("CREATE TABLE SI_VI_AD0 AS SELECT * FROM admatch0")
# conn.sql("CREATE TABLE SI_VI_AD1 AS SELECT * FROM admatch1")


for db in analyzed_rptr_dfs:
    print(db.loc[0,'Library'])
    
#     conn.sql("CREATE TABLE SI_VI_RPTR0 AS SELECT * FROM rptrmatch0")

RPTR_maxiprep_20X20_S11_L004_R1
RPTR_4_30_30X20_S207_L003
RPTR_4_30_16X16_0925_S14
RPTR_3_30_16X16_S15_L004
RPTR_4_15_20X20_S10_L004
RPTR_3_30_30X20_S207_L004
RPTR_4_30_16X16_sequences_R1


In [30]:
analyzed_rptr_dfs[0]

,index,BCs,TBB,Library,Putative_Tile,Putative_AD
0,TGTGGGCATGTTAG,11702164,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...,RPTR_maxiprep_20X20_S11_L004_R1,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...,GTATAATTCCA
1,TGGAGAAAGAAAAT,6525635,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0
2,ACTTCTAAATAAGC,3083395,CAATATAGAAGACAACAACCAGTTTCTCCAGCTGTTCAGAATCAAA...,RPTR_maxiprep_20X20_S11_L004_R1,CAATATAGAAGACAACAACCAGTTTCTCCAGCTGTTCAGAATCAAA...,TTTAGATTACA
3,CTCAATCGACAATT,2192286,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...,RPTR_maxiprep_20X20_S11_L004_R1,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...,GCCGACCCCTT
4,GTGATCGGTAACAC,980766,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0
...,...,...,...,...,...,...
27343,TCGCTTCCTCCCAC,1,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0
27344,TACCAGTGATAGTG,1,ACTGAGAATTGGTTTCCATTGTTTCCAGAATCTACTATTACTAATC...,RPTR_maxiprep_20X20_S11_L004_R1,ACTGAGAATTGGTTTCCATTGTTTCCAGAATCTACTATTACTAATC...,GAAGGCAGTTT
27345,GAAATAACAGGAAG,1,CATAGAAGGAAATCTTCTGTTAATGGTTCTCCAGTTACTAATGCTT...,RPTR_maxiprep_20X20_S11_L004_R1,CATAGAAGGAAATCTTCTGTTAATGGTTCTCCAGTTACTAATGCTT...,CCGCCATCACC
27346,TGTGGCAATGATAG,1,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0


In [61]:
# rstats_tocsv.to_csv('CiberVII_RPTRstats_TBB.csv')